# Cell Types Project
*Replace this with the name of your project*

## Team Member Names & Contributions

- **Kaushika Uppu (A16196379)**: Data wrangling, 
- **Emily Obedoza**: 

## Abstract

*Fill in your 3-4 sentence abstract here*

## Research Question

*Fill in your research question here*

# Background and Prior Work

*Fill in your background and prior work here (~500 words). You are welcome to use additional subheadings. You should also include a paragraph describing each dataset and how you'll be using them.* 

### References (include links):
(1)

(2)

## Hypothesis


*Fill in your hypotheses here*

## Setup

We first imported three packages that will be used throughout the project: `pandas`, `numpy`, `matplotlib.pyplot`, and `seaborn`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Electrophysiology Features of Mice and Human Cells

Our first dataset is from the Allen Cell Types database. This dataset is comprised of different cells, as well as information on those cells such as the brain cortical layer, dendrite type, and normalized depth. Since we are interested in comparing mouse and human cells, we split this data up into `mouse_cells` and `human_cells` by indexing by species using the CellTypesCache. We also set the index for both datasets to be `id` so that they can be merged with the electrophysiology data later on.

In [2]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

mouse_cells = pd.DataFrame(ctc.get_cells(species=['Mus musculus'])).set_index('id')
human_cells = pd.DataFrame(ctc.get_cells(species=['Homo Sapiens'])).set_index('id')

Some of the columns in this dataset are not needed for this project (for example, the `species` column is redundant after splitting the data by species), so we subsetted the dataframe to only contain columns with information that are relevant.

In [3]:
mouse_cells = mouse_cells.drop(columns=['reporter_status', 'cell_soma_location', 'species', 'disease_state', 'reconstruction_type', 'donor_id'])
human_cells = human_cells.drop(columns=['reporter_status', 'cell_soma_location', 'species', 'disease_state', 'reconstruction_type', 'donor_id'])

A look at the `mouse_cells` data:

In [4]:
mouse_cells.head(5)

,name,structure_layer_name,structure_area_id,structure_area_abbrev,transgenic_line,dendrite_type,apical,structure_hemisphere,normalized_depth
id,,,,,,,,,
565871768,Oxtr-2A-Cre;Ai14-293837.03.01.02,5,385,VISp,Oxtr-T2A-Cre,aspiny,NA,left,0.582057
469801138,Pvalb-IRES-Cre;Ai14-170927.05.02.01,4,385,VISp,Pvalb-IRES-Cre,aspiny,NA,right,0.469578
605889373,Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-337419.0...,2/3,385,VISp,Slc32a1-T2A-FlpO|Vipr2-IRES2-Cre,aspiny,NA,right,NaN
485909730,Cux2-CreERT2;Ai14-205530.03.02.01,5,385,VISp,Cux2-CreERT2,spiny,intact,right,0.478344
323865917,Scnn1a-Tg3-Cre;Ai14-172530.06.01.01,5,385,VISp,Scnn1a-Tg3-Cre,spiny,intact,left,0.608590


A look at the `human_cells` data:

In [5]:
human_cells.head(5)

,name,structure_layer_name,structure_area_id,structure_area_abbrev,transgenic_line,dendrite_type,apical,structure_hemisphere,normalized_depth
id,,,,,,,,,
525011903,H16.03.003.01.14.02,3,12113,FroL,,spiny,intact,right,NaN
528642047,H16.06.009.01.02.06.05,5,12141,MTG,,aspiny,NA,left,NaN
537256313,H16.03.006.01.05.02,4,12141,MTG,,spiny,truncated,right,NaN
519832676,H16.03.001.01.09.01,3,12141,MTG,,spiny,truncated,left,0.290951
596020931,H17.06.009.11.04.02,4,12141,MTG,,aspiny,NA,left,0.497825


We then needed to get the electrophysiology data for the cells. We set the index to be `specimen_id`, which is similar to `id` from the metadata, making it easier to merge the two. As seen below, this dataset contains various electrophysiology features for the different cells.

In [6]:
ephys_features = pd.DataFrame(ctc.get_ephys_features()).set_index('specimen_id')
ephys_features.head(5)

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_t_ramp,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest
specimen_id,,,,,,,,,,,,,,,,,,,,,
529878215,NaN,134.700000,22.697498,8.335459e-02,1.187680,13.295200,1.025916,-56.375004,-57.385420,-57.431251,...,13.295680,1.134780,-56.593754,-57.739586,-74.143753,3.029695,3.061646,2.969821,-80.468750,-73.553391
548459652,NaN,NaN,-24.887498,-3.913630e-19,1.099840,20.650105,1.025460,-54.000000,-54.828129,-54.656254,...,20.650735,1.160940,-55.406254,-55.242191,-73.500000,2.441895,2.245653,2.231575,-84.406258,-73.056595
579978640,0.009770,39.044800,-46.765002,5.267857e-01,1.157840,2.551310,1.025387,-59.500000,-58.234378,-59.940975,...,2.551960,1.089851,-60.062500,-58.570314,-61.371531,2.023762,2.162878,2.006406,-93.375008,-60.277321
439024551,-0.007898,117.816429,5.996250,1.542553e-01,1.989165,9.572025,1.028733,-47.531250,-50.359375,-65.500000,...,9.576308,1.423229,-49.406254,-52.718752,-75.273443,3.105931,3.491663,1.733896,-87.656250,-75.205559
515188639,0.022842,68.321429,14.910000,1.714041e-01,1.081980,2.462880,1.025620,-48.437504,-46.520837,-51.406253,...,2.490433,1.479690,-53.000004,-54.645837,-64.250003,3.285760,3.363504,4.234701,-81.625008,-63.474991


We then joined the ephys data with the metadata, separated by species. This allowed us to create two new dataframes, `mouse_df` and `human_df`, that contain all of the electrophysiology properties for these cells.

In [7]:
mouse_df = mouse_cells.join(ephys_features)
human_df = human_cells.join(ephys_features)

A glimpse of `mouse_df`:

In [8]:
mouse_df.head(5)

,name,structure_layer_name,structure_area_id,structure_area_abbrev,transgenic_line,dendrite_type,apical,structure_hemisphere,normalized_depth,adaptation,...,trough_t_ramp,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest
id,,,,,,,,,,,,,,,,,,,,,
565871768,Oxtr-2A-Cre;Ai14-293837.03.01.02,5,385,VISp,Oxtr-T2A-Cre,aspiny,NA,left,0.582057,NaN,...,14.738000,1.391268,-59.281254,-57.468754,-75.756252,1.564027,1.304349,1.679550,-87.906258,-74.926987
469801138,Pvalb-IRES-Cre;Ai14-170927.05.02.01,4,385,VISp,Pvalb-IRES-Cre,aspiny,NA,right,0.469578,0.000643,...,11.763808,1.290815,-55.875000,-52.515627,-69.109379,1.162618,1.197155,1.369171,-80.156250,-72.042976
605889373,Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-337419.0...,2/3,385,VISp,Slc32a1-T2A-FlpO|Vipr2-IRES2-Cre,aspiny,NA,right,NaN,-0.015098,...,8.432940,1.315510,-48.187500,-54.364586,-72.640628,3.379321,4.108774,2.680139,-83.593758,-72.712036
485909730,Cux2-CreERT2;Ai14-205530.03.02.01,5,385,VISp,Cux2-CreERT2,spiny,intact,right,0.478344,0.032340,...,2.888133,1.520193,-54.031254,-57.385419,-77.750005,3.042933,3.517684,3.274181,-101.000000,-76.928391
323865917,Scnn1a-Tg3-Cre;Ai14-172530.06.01.01,5,385,VISp,Scnn1a-Tg3-Cre,spiny,intact,left,0.608590,0.026732,...,3.467847,1.317042,-57.281254,-56.895833,-70.218751,2.974194,3.156117,2.946463,-88.406250,-69.402855


A glimpse of `human_df`:

In [9]:
human_df.head(5)

,name,structure_layer_name,structure_area_id,structure_area_abbrev,transgenic_line,dendrite_type,apical,structure_hemisphere,normalized_depth,adaptation,...,trough_t_ramp,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest
id,,,,,,,,,,,,,,,,,,,,,
525011903,H16.03.003.01.14.02,3,12113,FroL,,spiny,intact,right,NaN,0.027846,...,4.134987,1.375253,-53.968754,-59.510420,-71.197919,2.895461,2.559876,3.099787,-88.843758,-70.561035
528642047,H16.06.009.01.02.06.05,5,12141,MTG,,aspiny,NA,left,NaN,0.001610,...,NaN,1.051160,-67.468758,NaN,-70.875002,1.891881,NaN,1.989616,-101.000000,-69.209610
537256313,H16.03.006.01.05.02,4,12141,MTG,,spiny,truncated,right,NaN,0.054784,...,5.694547,1.389900,-52.125004,-51.520836,-72.900002,3.121182,3.464528,3.054681,-87.531250,-72.628105
519832676,H16.03.001.01.09.01,3,12141,MTG,,spiny,truncated,left,0.290951,0.179194,...,9.962780,1.211020,-53.875004,-52.416668,-73.693753,4.574865,3.817988,4.980603,-84.218758,-72.547661
596020931,H17.06.009.11.04.02,4,12141,MTG,,aspiny,NA,left,0.497825,0.196552,...,14.667340,1.336668,-63.593754,-63.239583,-75.518753,1.452890,1.441754,1.556087,-82.531250,-74.260269


### Gene Expression in Mice and Humans

Since we aim to compare gene expression between mice and humans, we then needed to get gene expression data for both species. Both datasets are from the Allen Brain Map RNA-Seq Data. For the mice gene expression data, we used the `Mouse Whole Cortex and Hippocampus 10x` dataset, while for the human gene expression data, we used the `Human Mulitple Cortical Areas SMART-seq` data. For both species, we used the `medians` files, which contain median expression by cell type.

In [23]:
mouse_genes = pd.read_csv('mouse_genes.csv').set_index('feature')
mouse_genes.head(5)

,108_Pvalb,229_L6 IT CTX,26_Ntng1 HPF,67_Sst,6_Lamp5 Lhx6,111_Pvalb,359_CA2-IG-FC,238_Car3,347_CA1-do,219_L6 IT CTX,...,50_Vip,190_L4/5 IT CTX,207_L5/6 IT CTX,342_CA1,260_L5 PT CTX,324_ProS,178_L4 IT CTX,284_L6 CT CTX,377_Astro,298_L6 CT ENT
feature,,,,,,,,,,,,,,,,,,,,,
Xkr4,9.92,9.08,9.21,10.27,9.68,10.03,8.515,8.14,8.92,9.16,...,9.35,8.86,8.96,9.8,7.41,7.83,8.22,8.9,0.0,9.09
Gm1992,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00
Gm37381,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00
Rp1,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00
Sox17,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00


In [11]:
human_genes = pd.read_csv('human_genes.csv').set_index('feature')
human_genes.head(5)

,Exc L5-6 FEZF2 ANKRD20A1,Exc L5-6 THEMIS TMEM233,Inh L1 LAMP5 NDNF,Exc L6 FEZF2 CPZ,Astro L1 FGFR3 MT1G,Exc L2-3 LINC00507 RPL9P17,Exc L6 FEZF2 KRT17,Exc L3-4 RORB FOLH1B,Endo L2-5 CLDN5,Exc L5-6 RORB LINC00320,...,Exc L4 RORB CACNG5,Exc L6 FEZF2 FAM95C,Inh L1-3 VIP GGH,Exc L3-5 FEZF2 DCN,Exc L6 FEZF2 ETV4,Exc L5-6 FEZF2 RSAD2,Inh L1 ADARB2 DISP2,Inh L5-6 LAMP5 SFTA3,Exc L6 FEZF2 TBC1D26,Inh L4-5 PVALB TRIM67
feature,,,,,,,,,,,,,,,,,,,,,
3.8-1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.8-1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.8-1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.8-1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5-HT3C2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since we want to look specifically at genes affecting GABA production, we then filtered both datasets for genes containing `GAD`, which previous research shows is involved in catalyzing the production of GABA from glutamate. We  then proceeded to get rid of any columns (cell types) that had zeros for every gene.

In [27]:
# Finding all GAD genes in the mice data
gad_mice = []
for i in mouse_genes.index:
    if 'Gad' in i:
        gad_mice.append(i)

# Filtering original gene expression dataset for only GAD genes
mouse_gad_genes = mouse_genes.loc[gad_mice]

# Dropping columns that had all zeros
mouse_gad_genes = mouse_gad_genes.loc[:, (mouse_gad_genes != 0).any(axis=0)]

mouse_gad_genes

,108_Pvalb,26_Ntng1 HPF,67_Sst,6_Lamp5 Lhx6,111_Pvalb,30_Ntng1 HPF,102_Sst HPF,117_Pvalb,123_Pvalb Vipr2,66_Sst,...,90_Sst,248_L5 PT CTX,216_L5/6 IT TPE-ENT,81_Sst,39_Sncg,99_Sst,72_Sst,288_L6 CT CTX,50_Vip,260_L5 PT CTX
feature,,,,,,,,,,,,,,,,,,,,,
Gad2,10.545,10.26,9.58,11.39,9.94,9.41,9.38,9.49,8.71,9.03,...,10.36,0.00,0.000,10.00,10.19,9.70,9.79,0.00,8.21,0.00
Gad1os,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Gad1,9.615,10.85,8.32,11.07,9.89,9.59,10.59,10.30,10.07,8.88,...,8.12,0.00,0.000,8.26,9.39,8.84,8.33,0.00,9.09,0.00
Gadd45a,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,7.07,0.00,0.00
Gadd45b,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Gadd45gip1,0.000,0.00,0.00,0.00,0.00,5.89,0.00,0.00,0.00,0.00,...,0.00,5.15,4.885,0.00,5.44,0.00,0.00,0.00,0.00,5.29
Gadl1,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Gadd45g,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [28]:
# Finding all GAD genes in the human data
gad_human = []
for i in human_genes.index:
    if 'GAD' in i:
        gad_human.append(i)

# Filtering original gene expression dataset for only GAD genes
human_gad_genes = human_genes.loc[gad_human]

# Dropping columns that had all zeros
human_gad_genes = human_gad_genes.loc[:, (human_gad_genes != 0).any(axis=0)]

human_gad_genes

,Inh L1 LAMP5 NDNF,Inh L3-6 PVALB MFI2,Inh L1-4 VIP CHRNA2,Inh L1-6 PVALB SCUBE3,Inh L5-6 PVALB STON2,Inh L4-6 SST MTHFD2P6,Inh L1-3 VIP SSTR1,Inh L3-6 VIP KCTD13,Inh L5-6 SST KLHL14,Unnamed: 26,...,Inh L1 VIP PRSS8,Inh L1-3 VIP CCDC184,Inh L1-3 PAX6 NABP1,Inh L1-5 VIP KCNJ2,Inh L2-4 VIP LGI2,Inh L1-4 LAMP5 DUSP4,Inh L1-3 VIP GGH,Inh L1 ADARB2 DISP2,Inh L5-6 LAMP5 SFTA3,Inh L4-5 PVALB TRIM67
feature,,,,,,,,,,,,,,,,,,,,,
GAD1,9.70,8.12,9.54,8.03,9.47,9.45,8.725,9.39,8.11,6.96,...,8.755,9.32,9.225,9.49,9.17,9.56,8.88,8.73,8.94,9.78
GAD2,9.62,0.00,5.86,0.00,8.13,7.70,7.810,5.84,7.66,1.18,...,7.255,5.48,5.800,6.93,6.40,9.36,4.37,7.14,9.61,8.36
GADD45A,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GADD45AP1,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GADD45B,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GADD45G,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GADD45GIP1,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GADL1,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ITGAD,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Summary

We have loaded in datasets for electrophyiology properties and gene expressions for both human and mice. We filtered out the gene expression datasets to only contain the genes that we are interested in for this project: `GAD` genes.

## Data Analysis & Results

In [15]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

## Conclusion & Discussion

*Fill in your discussion information here*